In [20]:
import pandas as pd
import nltk
import spacy
import gensim
import seaborn as sb
from gensim import corpora, models, similarities
from spacy.lang.en import English
from nltk.corpus import wordnet as wn

In [21]:
data=pd.read_csv("DurhamCity_original_data.csv",)

In [22]:
data.head()

,Unnamed: 0,page_number,word
0,1,1,fiscal
1,2,1,year
2,3,1,adopted
3,4,1,budget
4,5,1,july


In [23]:
data.drop(data.columns[0], axis=1)

,page_number,word
0,1,fiscal
1,1,year
2,1,adopted
3,1,budget
4,1,july
5,1,june
6,2,city
7,2,of
8,2,durham
9,2,north


In [24]:
data.describe()

,Unnamed: 0,page_number
count,81524.00000,81524.000000
mean,40762.50000,192.937393
std,23534.09601,108.890666
min,1.00000,1.000000
25%,20381.75000,108.000000
50%,40762.50000,202.000000
75%,61143.25000,276.000000
max,81524.00000,398.000000


In [25]:
#cleaning 
spacy.load('en')
parser=English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.orth_.startswith('http'):
            lda_tokens.append('com')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [26]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\messi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
def get_lemma(word):
    lemma=wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [28]:
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [29]:
#filtering stopwords
nltk.download('stopwords')
stop=set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\messi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
#preparing for lda
def prepare_lda(text):
    tokens=tokenize(text)
    tokens=[token for token in tokens if len(token)>4]
    tokens=[token for token in tokens if token not in stop]
    tokens=[get_lemma(token) for token in tokens]
    return tokens

In [31]:
#creating tokens
import random
text_data=[]
with open('GuilfordCounty_original_data.csv') as f:
    for line in f:
        tokens=prepare_lda(line)
        if random.random()>.99:
            print(tokens)
            text_data.append(tokens)

['32",2,"ava']
['280",3,"contest']
['369",4,"phillips']
['439",5,"governmental']
['589",7,"projections']
['627",7,"education']
['697",8,"services']
['786",9,"protection']
['961",11,"guidelines']
['983",11,"clerk']
['1029",11,"up']
['1150",11,"strengthen']
['1245",12,"incomes']
['1382",12,"what']
['1477",12,"and']
['1528",12,"reduction']
['2047",14,"rest']
['2135",15,"nutrition']
['2140",15,"children']
['2315",15,"revenue']
['2320",15,"based']
['2341",15,"entities']
['2739",16,"includes']
['2810",16,"medical']
['2977",17,"this']
['3211",18,"the']
['3281",18,"years']
['3525",18,"guilford']
['3606",18,"about']
['3809",19,"to']
['3845",19,"that']
['3959",19,"and']
['3984",19,"total']
['3987",19,"public']
['4335",20,"budget']
['4436",20,"positions']
['4468",21,"organizational']
['4476",21,"level']
['4525",21,"infrastructure']
['4532",21,"departments']
['4618",21,"high']
['4715",21,"software']
['4828",21,"renovate']
['4988",22,"infrastructure']
['5027",22,"several']
['5082",22,"in']
['5167",

['40569",185,"customer']
['40681",185,"layer']
['40728",186,"services']
['40886",187,"hour']
['40899",187,"of']
['40966",187,"bureau']
['41032",187,"recognized']
['41090",187,"reentry']
['41212",187,"practices']
['41276",188,"within']
['41357",188,"in']
['41521",188,"by']
['41787",189,"grant']
['42217",191,"agencies']
['42309",191,"of']
['42670",193,"county']
['42740",194,"opportunities']
['42884",195,"week']
['42973",195,"new']
['42982",196,"of']
['43100",196,"to']
['43225",196,"officer']
['43250",197,"duty']
['43415",199,"solid']
['43499",199,"summary']
['43566",199,"and']
['43585",199,"building']
['43806",201,"greensboro']
['43821",201,"quality']
['43985",202,"families']
['43993",202,"resources']
['44161",202,"focus']
['44407",203,"st']
['44442",203,"or']
['44513",203,"county']
['44742",204,"to']
['44759",204,"with']
['44845",204,"opportunity']
['45074",205,"schools']
['45292",206,"throughout']
['45308",206,"passive']
['45345",207,"with']
['45397",207,"and']
['45512",207,"levels']
[

['77172",361,"law']
['77438",362,"to']
['77450",362,"lot']
['77458",362,"for']
['77737",364,"were']
['77991",364,"juvenile']
['78120",366,"total']
['78296",369,"additl']
['78316",369,"a']
['78368",369,"is']
['78428",370,"park']
['78477",370,"for']
['78622",370,"park']
['78660",371,"name']
['78725",371,"wing']
['78801",371,"no']
['78818",371,"total']
['78832",372,"request']
['78933",372,"complete']
['79002",372,"fy']
['79042",372,"grant']
['79054",372,"estimated']
['79081",373,"code']
['79148",373,"total']
['79211",374,"life']
['79296",374,"for']
['79324",374,"include']
['79460",375,"and']
['79537",376,"cip']
['79578",376,"the']
['79628",376,"configured']
['79656",376,"construction']
['79662",376,"bonds']
['79771",377,"additional']
['79838",378,"description']
['79929",378,"professional']
['79959",379,"date']
['80060",379,"touchscreen']
['80111",379,"an']
['80123",379,"an']
['80160",379,"elections']
['80209",381,"those']
['80779",383,"traff']
['80820",384,"revenue']
['81090",385,"mainten

In [32]:
#creating dictionary
from gensim import corpora
dictionary=corpora.Dictionary(text_data)
corpus= [dictionary.doc2bow(text) for text in text_data]
import pickle 
pickle.dump(corpus, open('corpus.pkl', 'wb'))

In [33]:
#example
ldamodel=gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=6)
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} word: {}'.format(idx, topic))

Topic: 0 word: 0.006*"24637",116,"to" + 0.006*"71276",326,"parking" + 0.006*"40065",182,"for" + 0.006*"85700",413,"day" + 0.006*"85568",412,"based" + 0.006*"22418",103,"needed" + 0.006*"65147",287,"business" + 0.006*"70617",321,"greensboro" + 0.006*"44742",204,"to" + 0.006*"7680",33,"near"
Topic: 1 word: 0.006*"32227",150,"staff" + 0.006*"91597",471,"lots" + 0.006*"53316",240,"planning" + 0.006*"73018",337,"maintenance" + 0.006*"26742",126,"ecivis" + 0.006*"73847",343,"operating" + 0.006*"90091",460,"the" + 0.006*"78296",369,"additl" + 0.006*"78933",372,"complete" + 0.006*"65012",286,"and"
Topic: 2 word: 0.006*"20006",94,"sales" + 0.006*"68713",311,"are" + 0.006*"58290",259,"percent" + 0.006*"57948",258,"as" + 0.006*"86511",430,"performing" + 0.006*"93765",489,"curriculum" + 0.006*"36922",170,"with" + 0.006*"43499",199,"summary" + 0.006*"56220",251,"the" + 0.006*"10066",40,"ordinance"
Topic: 3 word: 0.006*"75786",353,"project" + 0.006*"9697",39,"general" + 0.006*"2739",16,"includes" + 

In [34]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)